In [2]:
%pip install pypdf -q
%pip install faiss-cpu -q
! pip -q install smolagents
! pip -q install litellm
!pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
# The other imports are likely correct and can remain as they are:
from langchain_openai import OpenAIEmbeddings
from langchain_openai.llms import OpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

In [5]:
loader = PyPDFLoader("Generalised_Learning_Path.pdf")
pages = loader.load()

In [6]:
for page in pages:
    print(page.page_content)


Month1: PythonFundamentals+IntroductiontoML
Week1:PythonFoundations&DataStructures
○ LearnPython■ IntroductiontoPython■ CompleteTutorial: LearnDataSciencePythonfromScratch■ HowtoLearnPythonStep-by-Step■ IntroductiontoDataScience(FreeCourse)○ Control Flow■ Pythoncontrol flow■ MCQsonPythonControl Flow(If StatementsandLoops)■ LoopsandControl Statements(In-DepthTutorial)○ Functions■ Functionsinpython■ MCQsonPythonFunctions○ DataStructures■ DataStructuresinPython■ GuidetoDataStructuresinPython■ SetsinPython■ HowtoSort PythonDictionaries■ MCQsonPythonSetsandOperations■ TuplesinPython■ MCQsonPythonTupleManipulation○ Classes&Objects■ AdvancedPython■ EverythingAbout ClassesandObjectsinPython
Week2:LibrariesinPython(Pandas, Numpy, Visualization)
○ Pandas■ PandasTipsandTricks■ UltimateGuidetoPandasforDataScience
■ PandasFunctionsforDataAnalysisandManipulation○ Scikit-Learn:OverviewandConcepts■ AComprehensiveGuidetoSklearn-Part 1: Overviewof thePackage■ Scikit-Learn: Objects-Fit vsTransformvsFit_

In [7]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
splitted_docs = splitter.split_documents(pages) # split_document accepts a list of documents

In [8]:
print(len(splitted_docs))
print(splitted_docs[0])
print(splitted_docs[1])
print(splitted_docs[2])

16
page_content='Month1: PythonFundamentals+IntroductiontoML
Week1:PythonFoundations&DataStructures
○ LearnPython■ IntroductiontoPython■ CompleteTutorial: LearnDataSciencePythonfromScratch■ HowtoLearnPythonStep-by-Step■ IntroductiontoDataScience(FreeCourse)○ Control Flow■ Pythoncontrol flow■ MCQsonPythonControl Flow(If StatementsandLoops)■ LoopsandControl Statements(In-DepthTutorial)○ Functions■ Functionsinpython■ MCQsonPythonFunctions○ DataStructures■ DataStructuresinPython■ GuidetoDataStructuresinPython■ SetsinPython■ HowtoSort PythonDictionaries■ MCQsonPythonSetsandOperations■ TuplesinPython■ MCQsonPythonTupleManipulation○ Classes&Objects■ AdvancedPython■ EverythingAbout ClassesandObjectsinPython
Week2:LibrariesinPython(Pandas, Numpy, Visualization)
○ Pandas■ PandasTipsandTricks■ UltimateGuidetoPandasforDataScience' metadata={'source': 'Generalised_Learning_Path.pdf', 'page': 1}
page_content='■ PandasFunctionsforDataAnalysisandManipulation○ Scikit-Learn:OverviewandConcepts■ ACompreh

In [10]:
# Set the API key for OpenAIEmbeddings
embed_model = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])
# Generate embeddings for the documents
embeddings = embed_model.embed_documents([chunk.page_content for chunk in splitted_docs])
print(f"Embeddings shape: {len(embeddings), len(embeddings[0])}")

Embeddings shape: (16, 1536)


In [11]:
vector_db = FAISS.from_documents(
    documents = splitted_docs,
    embedding = embed_model
)

In [12]:
similar_docs = vector_db.similarity_search("what is the objective for data sharing with donors?", k=5)
print(similar_docs[0].page_content)

○ Prompt EngineeringConceptsforStableDiffusion
● OtherDiffusion-BasedTools○ MidJourney○ UnderstandingDALL·E3
● DiffusionModels○ StableDiffusion3○ HowtoGenerateandEdit DALL·EImagesinCopilot
● DiffusionModels:CoreComponentsandProcesses○ Different Componentsof DiffusionModels○ Positional EncodinginStableDiffusion○ NoiseSchedulesinStableDiffusion○ ReverseDiffusionProcess○ ForwardProcessinStableDiffusion
StableDiffusionandImageGeneration(COURSES)
● ExploringStability.AI● MidJourney: FromInspirationtoImplementation● NanoCourse: Dreambooth-StableDiffusionforCustomImages
Week22-23:Multimodal LLMsandAgenticAI
● Multimodal LLMs○ LLaMA3.290BvsGPT-4O○ Multimodal Chatbot withGPT-4O○ BuildinganImageDataExtractorUsingGemini VisionLLM○ Pixtral 12B
● AgenticAI○ LangChain'sAgent Framework○ BuildAI AgentsfromScratch○ LLaMAAgents: AgentsasaService○ AgenticWorkflowwithCreWAI andGroq○ BuildanAI CodingAgent withLangGraphbyLangChain○ BuildingSmart AI AgentswithLangChain


In [16]:
from smolagents import Tool
class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of the documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
            }
        }
    output_type = "string"
    
    def __init__(self, vector_db, **kwargs): # Add vector_db as an argument
        super().__init__(**kwargs)
        self.vector_db = vector_db # Store the vector database

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"
        docs = self.vector_db.similarity_search(query, k=4) # Perform search here
        return "\nRetrieved documents:\n" + "".join(
            [f"\n\n===== Document {str(i)} =====\n" + doc.page_content for i, doc in enumerate(docs)]
            )

/home/kali/Downloads/PythonProjects/AiAgents/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:

retriever_tool = RetrieverTool(vector_db=vector_db) # Pass vector_db during instantiation

In [ ]:
from smolagents import LiteLLMModel, DuckDuckGoSearchTool
model = LiteLLMModel(model_id="gpt-4o", api_key = "your_api_key")
search_tool = DuckDuckGoSearchTool()

In [ ]:
from smolagents import HfApiModel, CodeAgent
agent = CodeAgent(
    tools=[retriever_tool,search_tool], model=model, max_steps=6, verbose=True
)

In [ ]:
agent.run("Tell me about Analytics Vidhya")

In [ ]:
agent_output = agent.run("what are the constraints for data sharing with donors?")
print("Final output:")
print(agent_output)

In [ ]:
agent_output1 = agent.run("WHAT ARE THE OBJECTIVES FOR DATA SHARING WITH DONORS?")